In [1]:
import mhcflurry, seaborn, numpy, pandas, pickle, sklearn, collections, scipy
import mhcflurry.data
import sklearn.metrics
import sklearn.cross_validation

/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")
/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Couldn't import dot_parser, loading of dot files will not be possible.


/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
data = mhcflurry.data.load_allele_datasets("/Users/tim/sinai/git/mhcflurry/bdata.2009.mhci.public.1.txt")

In [3]:
a = data["HLA-A0201"]
a.X_index[0]

array([ 0,  0,  0,  0, 16,  1,  0,  9, 17])

In [21]:
def projections(peptide, edit_distance=1):
    if edit_distance == 0:
        return set([peptide])
    return set.union(*[
            projections(p, edit_distance - 1)
            for p in (peptide[0:i] + "." + peptide[(i+1):] for i in range(len(peptide)))
    ])

def edit_distance_subcomponent_labels(peptides, edit_distance=1):
    adjacency_matrix = scipy.sparse.dok_matrix((len(peptides), len(peptides)), dtype=bool)
    projection_to_peptide_indices = collections.defaultdict(set)
    for (i, peptide) in enumerate(peptides):
        for projection in projections(peptide, edit_distance=edit_distance):
            projection_to_peptide_indices[projection].add(i)
    for indices in projection_to_peptide_indices.values():
        for index1 in indices:
            for index2 in indices:
                adjacency_matrix[index1, index2] = True
    
    (num_components, labels) = scipy.sparse.csgraph.connected_components(adjacency_matrix, directed=False)
    return labels

def balanced_folds(labels, num_folds=2):
    # Simple greedy approximation to balanced partition problem
    folds = [set() for i in range(num_folds)]
    df = pandas.Series(labels).to_frame().reset_index()
    df.columns = ["point", "label"]
    grouped = df.groupby("label").apply(lambda group: list(group.point)).to_frame()
    grouped.columns = ["indices"]
    grouped["count"] = [len(x) for x in grouped.indices]
    grouped.sort("count", ascending=False, inplace=True)
    for (label, row) in grouped.iterrows():
        min_fold = folds[numpy.argmin([len(x) for x in folds])]
        min_fold.update(row.indices)
    
    return [
        numpy.array(sorted(fold))
        for fold in folds
    ]
    
def cv_indices(peptides, edit_distance=1, num_folds=3, shuffle=True):
    if shuffle:
        def maybe_shuffle(indices):
            return numpy.random.permutation(indices)
    else:
        def maybe_shuffle(indices):
            return indices
    
    labels = edit_distance_subcomponent_labels(peptides, edit_distance=edit_distance)
    folds = balanced_folds(labels, num_folds=num_folds)
    all_indices = set(numpy.arange(len(peptides)))
    for test_fold in folds:
        train_fold = numpy.array(list(all_indices.difference(test_fold)))
        yield (maybe_shuffle(train_fold), maybe_shuffle(test_fold))
        
    
%time labels = edit_distance_subcomponent_labels(a.peptides, 1)
%time folds = balanced_folds(labels, 5)
labels


CPU times: user 672 ms, sys: 31.6 ms, total: 704 ms
Wall time: 687 ms
CPU times: user 876 ms, sys: 10.9 ms, total: 887 ms
Wall time: 908 ms


array([   0,    1,    2, ..., 6652, 6653, 6654], dtype=int32)

In [27]:
%time labels = edit_distance_subcomponent_labels(a.peptides, 0)
labels.max()
labels.shape


CPU times: user 95.5 ms, sys: 22.3 ms, total: 118 ms
Wall time: 111 ms


(6961,)

In [22]:
for (train, test) in cv_indices(a.peptides, edit_distance=0):
    print(train, test)
    

(array([3244, 4894, 6646, ..., 6832,  780,  692]), array([ 767, 3387, 1328, ..., 4490, 3659, 1911]))
(array([4220, 5170,  874, ...,  454, 1144, 1778]), array([3446,  997, 3655, ..., 4569, 1844, 3468]))
(array([3549, 1279, 2552, ..., 2142, 2311,  675]), array([2566, 6411, 4595, ..., 2731, 2938, 1601]))


In [16]:
reload(mhcflurry)
reload(mhcflurry.class1_binding_predictor)

<module 'mhcflurry.class1_binding_predictor' from '/Users/tim/sinai/git/mhcflurry/mhcflurry/class1_binding_predictor.py'>

In [42]:
model = mhcflurry.class1_binding_predictor.Class1BindingPredictor(
    embedding_input_dim=21,
    embedding_output_dim=32,
    learning_rate=0.01,
    activation="tanh")

In [18]:
def roc_auc_ic50_threshold(y, y_pred, threshold_nm=500):
    return sklearn.metrics.roc_auc_score(
        model.log_to_ic50(y) < threshold_nm,
        model.log_to_ic50(y_pred) < threshold_nm)
    
scorer = sklearn.metrics.make_scorer(roc_auc_ic50_threshold)
sklearn.cross_validation.cross_val_score(model, a.X_index, a.Y, scorer, cv_indices(a.peptides))

array([ 0.86992363,  0.84365301,  0.83955602])

In [28]:
sklearn.cross_validation.cross_val_score(model, a.X_index, a.Y, scorer, cv_indices(a.peptides, edit_distance=0))

array([ 0.84956871,  0.87736409,  0.86106563])

In [29]:
sklearn.cross_validation.cross_val_score(model, a.X_index, a.Y, scorer, cv_indices(a.peptides, edit_distance=2))

array([ 0.8549138 ,  0.85409191,  0.87046823])

In [36]:
data["Gogo-B0101"].Y.shape

(14,)

In [43]:
result = {}
for allele in list(sorted(data))[:10]:
    values = data[allele]
    result[allele] = sklearn.cross_validation.cross_val_score(
        model, values.X_index, values.Y, scorer, cv_indices(values.peptides))
    print(allele, result[allele])
    
for (key, values) in result.iteritems():
    print(key, values)
    

('Gogo-B0101', array([ 0.35,  0.25,  0.5 ]))
('H-2-DB', array([ 0.80445675,  0.84123684,  0.89734347]))
('H-2-DD', array([ 0.58387097,  0.60912698,  0.59325397]))
('H-2-KB', array([ 0.73026071,  0.68359534,  0.7560893 ]))
('H-2-KD', array([ 0.76336898,  0.70289855,  0.79196487]))
('H-2-KK', array([ 0.68577075,  0.67935484,  0.78709677]))
('H-2-LD', array([ 0.63095238,  0.86363636,  0.85151515]))
('HLA-A0101', array([ 0.82295861,  0.79946528,  0.86610991]))
('HLA-A0201', array([ 0.87802025,  0.86698864,  0.86856158]))
('HLA-A0202', array([ 0.84085982,  0.85036179,  0.8273601 ]))
('HLA-A0201', array([ 0.87802025,  0.86698864,  0.86856158]))
('HLA-A0202', array([ 0.84085982,  0.85036179,  0.8273601 ]))
('H-2-DB', array([ 0.80445675,  0.84123684,  0.89734347]))
('H-2-DD', array([ 0.58387097,  0.60912698,  0.59325397]))
('H-2-KK', array([ 0.68577075,  0.67935484,  0.78709677]))
('Gogo-B0101', array([ 0.35,  0.25,  0.5 ]))
('H-2-LD', array([ 0.63095238,  0.86363636,  0.85151515]))
('H-2-KB',

In [41]:
model = mhcflurry.class1_binding_predictor.Class1BindingPredictor(
    embedding_input_dim=21,
    embedding_output_dim=32,
    learning_rate=0.01,
    activation="tanh")

result = {}
for allele in list(sorted(data, key=lambda x: -1 * data[x].Y.shape[0]))[:10]:
    values = data[allele]
    #print(allele, values.Y.shape[0])
    result[allele] = sklearn.cross_validation.cross_val_score(
        model, values.X_index, values.Y, scorer, cv_indices(values.peptides))
    print(allele, result[allele])
    
for (key, values) in result.iteritems():
    print(key, values)

('HLA-A0201', 6961)
('HLA-A0201', array([ 0.88591598,  0.8921594 ,  0.8863821 ]))
('HLA-A0301', 4601)
('HLA-A0301', array([ 0.82211658,  0.84389135,  0.80404472]))
('HLA-A0203', 3937)
('HLA-A0203', array([ 0.87039776,  0.87301872,  0.85440652]))
('HLA-A1101', 3862)
('HLA-A1101', array([ 0.89471562,  0.81754575,  0.87795699]))
('HLA-A3101', 3309)
('HLA-A3101', array([ 0.84547391,  0.83338317,  0.80496805]))
('HLA-A0206', 3223)
('HLA-A0206', array([ 0.81704192,  0.82848796,  0.84186113]))
('HLA-A6802', 3188)
('HLA-A6802', array([ 0.82971764,  0.8122962 ,  0.79950669]))
('HLA-A0101', 3169)
('HLA-A0101', array([ 0.81362577,  0.78240669,  0.82866379]))
('HLA-B1501', 3142)
('HLA-B1501', array([ 0.81558246,  0.76136838,  0.80164387]))
('HLA-B0702', 2974)
('HLA-B0702', array([ 0.87536917,  0.85384887,  0.85821342]))
('HLA-A0206', array([ 0.81704192,  0.82848796,  0.84186113]))
('HLA-A1101', array([ 0.89471562,  0.81754575,  0.87795699]))
('HLA-A0203', array([ 0.87039776,  0.87301872,  0.854406

In [31]:
result

{'Gogo-B0101': array([ 0.35,  0.  ,  0.5 ]),
 'H-2-DB': array([ 0.80259192,  0.80323763,  0.88438937]),
 'H-2-DD': array([ 0.6       ,  0.48412698,  0.60912698]),
 'H-2-KB': array([ 0.74255326,  0.70298595,  0.71285589]),
 'H-2-KD': array([ 0.73729947,  0.67687747,  0.79033832]),
 'H-2-KK': array([ 0.63372859,  0.70322581,  0.72709677]),
 'H-2-LD': array([ 0.70238095,  0.79186603,  0.8       ]),
 'HLA-A0101': array([ 0.79496009,  0.77293741,  0.8203125 ]),
 'HLA-A0201': array([ 0.86748346,  0.85718596,  0.84801616])}